In [4]:
#Importing Libraries

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import CocoDetection
from torchvision import transforms
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt
from art.attacks.evasion import SquareAttack
from art.estimators.object_detection import PyTorchFasterRCNN
from art.utils import load_dataset, preprocess
from art.utils import to_categorical
from keras.preprocessing import image

In [6]:
#Loading the Yolov5 model

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()

/usr/local/lib/python3.8/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: YOLOv5 requirement "gitpython>=3.1.30" not found, attempting AutoUpdate...
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidi

  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [7]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
])

In [16]:
#loading the coco dataset

dataset = datasets.CocoDetection(root='dataset/val2017', annFile='instances_val2017.json', transform=transform)
dataset


Loading annotations into memory...
Done (t=0.68s)
Creating index...
index created!


Dataset CocoDetection
    Number of datapoints: 5000
    Root location: dataset/val2017
    StandardTransform
Transform: Compose(
               Resize(size=(416, 416), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )

In [17]:
#Defining the Adversarial Square Attack function

def adversarial_square_attack(model, images, labels, eps=0.01, alpha=0.01, iters=10):
    """
    Implements the Adversarial Square Attack on a PyTorch model.
    """
    images_adv = images.clone().detach()
    for i in range(iters):
        images_adv.requires_grad = True
        logits_adv = model(images_adv)
        loss = F.cross_entropy(logits_adv, labels)
        loss.backward()

        with torch.no_grad():
            images_adv += alpha * images_adv.grad.sign()
            images_adv = torch.max(torch.min(images_adv, images + eps), images - eps).clamp(0, 1)
            images_adv.grad.zero_()
    return images_adv

In [19]:
# Selecting a random image and label from the dataset
idx = random.randint(0, len(dataset)-1)
image, label = dataset[idx]


In [20]:
# Reshape the labels tensor to have a size of [1, num_classes]
#labels = torch.zeros(1, 85)
#labels[0, label[1]['category_id']-1] = 1



labels = torch.zeros(1, 85)
labels[0, label[1]['category_id']-1] = 1
labels = labels.unsqueeze(0).to('cuda') 
labels

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]]], device='cuda:0')

In [21]:
# Move the labels tensor to the GPU
labels = labels.to('cuda')
labels

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]]], device='cuda:0')

In [24]:
# Generate an adversarial example for the image and label

image_adv = adversarial_square_attack(model, image.unsqueeze(0).to('cuda'), labels.long().unsqueeze(0).to('cuda'))

RuntimeError: Expected target size [1, 85], got [1, 1, 1, 85]

In [25]:
# Converting the PyTorch tensor to a numpy array

image = image.permute(1, 2, 0).numpy()
image_adv = image_adv.squeeze().permute(1, 2, 0).numpy()

NameError: name 'image_adv' is not defined

In [ ]:
# Displaying the images

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.imshow(image)
ax1.set_title('Original Image')
ax2.imshow(image_adv)
ax2.set_title('Adversarial Example')
plt.show()